### Proposition
Retrieve each `SystemInformationID` along with its `Database Version` and `VersionDate`, and calculate both the rank and dense rank based on the `VersionDate`.

### Functional Specification

- **Query Name**: RetrieveRankedDatabaseVersions
- **Description**: This query selects the `SystemInformationID`, `Database Version`, and `VersionDate` from the `AWBuildVersion` table and computes two types of rankings: `RANK` and `DENSE_RANK`, based on the `VersionDate`. The `RANK` function allows for gaps in ranking if multiple rows share the same date, whereas `DENSE_RANK` assigns consecutive ranks for rows with identical `VersionDate`.
- **Inputs**: No specific input parameters required, as the query pulls all records from `AWBuildVersion`.
- **Outputs**:
  - `SystemInformationID`: Identifier of the system information.
  - `Database Version`: Version of the database.
  - `VersionDate`: Date associated with the database version.
  - `Rank`: Ranking of each row based on `VersionDate`, with gaps for ties.
  - `DenseRank`: Dense ranking of each row based on `VersionDate`, with consecutive ranking for ties.
- **Steps**:
  1. Retrieve columns `SystemInformationID`, `Database Version`, and `VersionDate` from the `AWBuildVersion` table.
  2. Apply the `RANK` and `DENSE_RANK` window functions, ordered by `VersionDate`, to calculate the rank values for each row.
- **Assumptions**: 
  - The `AWBuildVersion` table contains valid dates in `VersionDate`.
  - No filtering criteria are applied to the dataset; all records in `AWBuildVersion` are included in the output.

- **Example Output**:

| SystemInformationID | Database Version | VersionDate | Rank | DenseRank |
|---------------------|------------------|-------------|------|-----------|
| 1                   | 1.0.0.0          | 2022-01-01 | 1    | 1         |
| 2                   | 1.0.0.1          | 2022-02-01 | 2    | 2         |
| 3                   | 1.0.0.1          | 2022-02-01 | 2    | 2         |
| 4                   | 1.0.0.2          | 2022-03-01 | 4    | 3         |


In [13]:
use NewAdventureWorks2019
go
SELECT SystemInformationID, [Database Version], VersionDate,
       RANK() OVER(ORDER BY VersionDate) AS Rank,
       DENSE_RANK() OVER(ORDER BY VersionDate) AS DenseRank
FROM dbo.AWBuildVersion;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.004

SystemInformationID,Database Version,VersionDate,Rank,DenseRank
1,15.0.4280.7,2023-01-23 13:08:53.190,1,1


### Proposition
Count the number of log events in the `DatabaseLog` table for each year.

### Functional Specification

**Query Name**: AnnualLogEventCount

**Description**: This query retrieves the count of log events in the `DatabaseLog` table, grouped by year. It provides an overview of the number of log entries made in each year, which can be useful for analyzing the frequency of events over time.

**Inputs**: None required. The query will operate on all entries in the `DatabaseLog` table.

**Outputs**:
- `LogYear`: The year of the log event (derived from the `PostTime` column).
- `LogCount`: The total number of log events that occurred in each year.

**Steps**:
1. Extract the year from the `PostTime` column using the `YEAR` function and alias it as `LogYear`.
2. Count the number of log entries for each year.
3. Group the results by `LogYear` to produce the count of log entries per year.

**Assumptions**:
- The `PostTime` column contains valid datetime values for each log event.

**Example Output**:

| LogYear | LogCount |
| ------- | -------- |
| 2020    | 150      |
| 2021    | 175      |
| 2022    | 200      |


In [12]:
use NewAdventureWorks2019
go
SELECT YEAR(PostTime) AS LogYear, COUNT(*) AS LogCount
FROM dbo.DatabaseLog
GROUP BY YEAR(PostTime);

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.013

LogYear,LogCount
2023,1596


## Proposition
Retrieve the count of employees in each department based on their assigned shifts. The output should display each `DepartmentID` with separate columns for each shift (`Shift1`, `Shift2`, `Shift3`), showing the count of employees assigned to each shift.

## Functional Specification

### Query Name
Department Shift Count

### Description
This query displays the count of employees per shift for each department. It uses the `PIVOT` operator to transform rows into columns, showing the shift counts in separate columns for easier comparison across departments.

### Inputs
- **DepartmentID**: The unique identifier for each department in `EmployeeDepartmentHistory`.
- **ShiftID**: The unique identifier for each shift (1, 2, or 3).

### Outputs
- **DepartmentID**: The unique identifier for each department.
- **Shift1**: The count of employees in Shift 1 for the given department.
- **Shift2**: The count of employees in Shift 2 for the given department.
- **Shift3**: The count of employees in Shift 3 for the given department.

### Steps
1. Select `DepartmentID` and `ShiftID` from the `HumanResources.EmployeeDepartmentHistory` table.
2. Use the `PIVOT` operator to count the occurrences of each `ShiftID` per `DepartmentID`.
3. Transform `ShiftID` values into separate columns (`Shift1`, `Shift2`, `Shift3`) with counts as values.

### Assumptions
- The `ShiftID` column has a limited range (1, 2, 3).
- Each `DepartmentID` can have multiple employees in each shift, or none.
- The `EmployeeDepartmentHistory` table contains data for employee assignments to shifts.

### Example Output

| DepartmentID | Shift1 | Shift2 | Shift3 |
|--------------|--------|--------|--------|
| 1            | 5      | 3      | 2      |
| 2            | 10     | 4      | 7      |
| 3            | 0      | 6      | 1      |

In this example, Department 1 has 5 employees in Shift 1, 3 in Shift 2, and 2 in Shift 3. Department 2 has 10 in Shift 1, 4 in Shift 2, and 7 in Shift 3. Department 3 has no employees in Shift 1, 6 in Shift 2, and 1 in Shift 3.


In [11]:
use NewAdventureWorks2019
go
SELECT DepartmentID, [1] AS Shift1, [2] AS Shift2, [3] AS Shift3
FROM (SELECT DepartmentID, ShiftID FROM HumanResources.EmployeeDepartmentHistory) AS src
PIVOT (COUNT(ShiftID) FOR ShiftID IN ([1], [2], [3])) AS pvt;



Commands completed successfully.

(16 rows affected)

Total execution time: 00:00:00.008

DepartmentID,Shift1,Shift2,Shift3
1,7,0,0
2,4,0,0
3,18,0,0
4,10,0,0
5,13,0,0
6,4,0,0
7,80,54,46
8,4,1,1
9,6,0,0
10,11,0,0


## Proposition

Retrieve the earliest, latest, and average modification dates for departments within the `HumanResources.Department` table. The average modification date will be expressed as the average number of days since January 1, 2000.

## Functional Specification

### Query Name
DepartmentModificationDateStatistics

### Description
This query returns three key statistics regarding the modification dates of department records in the `HumanResources.Department` table:
1. The earliest (`MinDate`) modification date.
2. The latest (`MaxDate`) modification date.
3. The average number of days since January 1, 2000 (`AvgDaysSince2000`) as a float, representing the average modification date across all departments.

### Inputs
No external inputs are required for this query as it operates directly on the `HumanResources.Department` table data.

### Outputs
- **MinDate**: The minimum date in the `ModifiedDate` column, representing the earliest modification.
- **MaxDate**: The maximum date in the `ModifiedDate` column, representing the latest modification.
- **AvgDaysSince2000**: The average number of days since January 1, 2000, computed by calculating the difference in days between each `ModifiedDate` and January 1, 2000, then averaging the results.

### Steps
1. Select the minimum and maximum dates from the `ModifiedDate` column.
2. Use `DATEDIFF` to calculate the difference in days between each `ModifiedDate` and January 1, 2000.
3. Convert the day difference to a floating-point number to calculate an average.
4. Output `MinDate`, `MaxDate`, and `AvgDaysSince2000` as the results.

### Assumptions
- The `ModifiedDate` column in the `HumanResources.Department` table is populated with valid dates.
- The date '2000-01-01' is used as the baseline for calculating the average days since that point in time.

### Example Output

| MinDate    | MaxDate    | AvgDaysSince2000 |
|------------|------------|------------------|
| 2001-05-14 | 2023-09-10 | 4815.67          |

This output would indicate that the earliest department modification was on May 14, 2001, the latest was on September 10, 2023, and the average modification date is approximately 4,815.67 days after January 1, 2000.


In [10]:
use NewAdventureWorks2019
go
SELECT MIN(ModifiedDate) AS MinDate, MAX(ModifiedDate) AS MaxDate,
 AVG(CAST(DATEDIFF(day, '2000-01-01', ModifiedDate) AS FLOAT)) AS AvgDaysSince2000
FROM HumanResources.Department;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.010

MinDate,MaxDate,AvgDaysSince2000
2008-04-30 00:00:00.000,2008-04-30 00:00:00.000,3042


### Proposition
Identify pairs of addresses located in the same city from the `Person.Address` table.

### Functional Specification

- **Query Name**: FindMatchingAddressPairs
- **Description**: This query finds pairs of addresses that are located in the same city by performing a self-join on the `Person.Address` table.
- **Inputs**: `Person.Address` table, specifically the `AddressID` and `City` columns.
- **Outputs**:
  - `Address1`: The `AddressID` of the first address in each matching pair.
  - `Address2`: The `AddressID` of the second address in each matching pair, where `Address2` is greater than `Address1` to avoid duplicate pairs.
  - `City`: The city where both addresses are located.
- **Steps**:
  1. Perform a self-join on the `Person.Address` table, where both aliases (`a1` and `a2`) have the same `City`.
  2. Ensure that `a1.AddressID` is less than `a2.AddressID` to avoid duplicating pairs in reverse order.
- **Assumptions**: 
  - Each address is uniquely identified by `AddressID`.
  - There are multiple addresses in the same city, allowing for pairs to be identified.
- **Example Output**:

| Address1 | Address2 | City      |
|----------|----------|-----------|
| 101      | 102      | Seattle   |
| 203      | 204      | New York  |
| 305      | 306      | Chicago   |

This output shows pairs of addresses (identified by `AddressID`) located in the same city.


In [9]:
use NewAdventureWorks2019
go
SELECT a1.AddressID AS Address1, a2.AddressID AS Address2, a1.City
FROM Person.Address AS a1
JOIN Person.Address AS a2 ON a1.City = a2.City AND a1.AddressID < a2.AddressID;

Commands completed successfully.

(1089658 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:02.211

Address1,Address2,City
458,532,Ottawa
31,532,Ottawa
483,532,Ottawa
495,532,Ottawa
456,497,Burnaby
445,497,Burnaby
488,497,Burnaby
26916,29781,Dunkerque
23032,29781,Dunkerque
19945,29781,Dunkerque


## Proposition
Retrieve the most recent database version and its associated version date from the `AWBuildVersion` table.

## Functional Specification

- **Query Name**: GetLatestDatabaseVersion
- **Description**: This query retrieves the latest database version and the date it was recorded from the `AWBuildVersion` table.
- **Inputs**: None
- **Outputs**:
  - `Database Version`: The latest version of the database as a string.
  - `VersionDate`: The date associated with the latest database version.
- **Steps**:
  1. Select the `Database Version` and `VersionDate` columns from the `AWBuildVersion` table.
  2. Order the records by `VersionDate` in descending order to ensure the latest version appears first.
  3. Limit the results to a single row to retrieve only the most recent version.

- **Assumptions**:
  - The `AWBuildVersion` table contains a history of database versions with corresponding dates.
  - `VersionDate` reliably indicates the order of version history, with the most recent version having the latest date.

- **Example Output**:

  | Database Version | VersionDate           |
  |------------------|-----------------------|
  | 15.0.2000.5      | 2023-11-12 14:30:00.0 |


In [8]:
use NewAdventureWorks2019
go
SELECT TOP 1 [Database Version], VersionDate
FROM dbo.AWBuildVersion
ORDER BY VersionDate DESC;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.005

Database Version,VersionDate
15.0.4280.7,2023-01-23 13:08:53.190


### Proposition

Retrieve a count of events logged by each user in the `DatabaseLog` table. This allows us to identify the frequency of actions performed by each `DatabaseUser`.

---

### Functional Specification

**Query Name:** EventCountByUser

**Description:** This query counts the number of log entries associated with each user in the `DatabaseLog` table, grouped by `DatabaseUser`.

**Inputs:**
- `DatabaseLog` table with relevant columns (`DatabaseUser` and event details).

**Outputs:**
- `DatabaseUser`: The name of the user who performed actions logged in the `DatabaseLog`.
- `EventCount`: The total number of events associated with each user.

**Steps:**
1. Select the `DatabaseUser` column from the `DatabaseLog` table.
2. Use `COUNT(*)` to calculate the total number of events for each `DatabaseUser`.
3. Group results by `DatabaseUser` to get individual counts per user.

**Assumptions:**
- The `DatabaseLog` table is populated with event data that includes `DatabaseUser` entries.

**Example Output:**

| DatabaseUser | EventCount |
|--------------|------------|
| User1        | 15         |
| User2        | 8          |
| User3        | 21         |

---

This query provides a summary of event activity for each user, which can be useful for monitoring or auditing purposes.


In [7]:
SELECT DatabaseUser, COUNT(*) AS EventCount
FROM dbo.DatabaseLog
GROUP BY DatabaseUser;

(1 row affected)

Total execution time: 00:00:00.008

DatabaseUser,EventCount
dbo,1596


### Proposition
Retrieve the names of departments along with the count of employees in each department.

### Functional Specification

- **Query Name**: DepartmentEmployeeCount
- **Description**: This query lists each department's name along with the total number of employees associated with it. It helps to understand the distribution of employees across different departments within the organization.
- **Inputs**:
  - `HumanResources.Department` table
  - `HumanResources.EmployeeDepartmentHistory` table
- **Outputs**:
  - `DepartmentName`: The name of the department.
  - `EmployeeCount`: The total number of employees in the department.
- **Steps**:
  1. Join the `HumanResources.Department` table with the `HumanResources.EmployeeDepartmentHistory` table on `DepartmentID`.
  2. Group the results by `DepartmentName`.
  3. Count the number of `BusinessEntityID` entries in each department to get the total employee count.
- **Assumptions**:
  - Each employee is associated with at least one department in the `EmployeeDepartmentHistory` table.
  - `DepartmentID` is a unique identifier for departments in both tables.
- **Example Output**:

| DepartmentName       | EmployeeCount |
|----------------------|---------------|
| Sales                | 25            |
| Engineering          | 40            |
| Human Resources      | 15            |
| Marketing            | 20            |
| Research and Development | 30       |


In [6]:
SELECT d.Name AS DepartmentName, COUNT(e.BusinessEntityID) AS EmployeeCount
FROM HumanResources.Department AS d
JOIN HumanResources.EmployeeDepartmentHistory AS e
ON d.DepartmentID = e.DepartmentID
GROUP BY d.Name;


(16 rows affected)

Total execution time: 00:00:00.016

DepartmentName,EmployeeCount
Engineering,7
Tool Design,4
Sales,18
Marketing,10
Purchasing,13
Research and Development,4
Production,180
Production Control,6
Human Resources,6
Finance,11


### Proposition
Retrieve each department's ID and name along with the count of employees assigned to it, and rank the departments based on the number of employees in descending order. The department with the most employees will have a rank of 1.

### Functional Specification

- **Query Name**: Department Employee Count and Ranking
- **Description**: This query returns each department's ID, name, total number of employees, and its rank based on employee count. Departments with the same employee count will receive the same rank.
- **Inputs**: None
- **Outputs**:
  - `DepartmentID`: Unique identifier for the department.
  - `Name`: The name of the department.
  - `EmployeeCount`: The total number of employees in each department.
  - `DepartmentRank`: Rank of each department based on `EmployeeCount`, in descending order.
- **Steps**:
  1. Join `HumanResources.Department` and `HumanResources.EmployeeDepartmentHistory` tables on `DepartmentID`.
  2. Count the number of employees (`BusinessEntityID`) associated with each department.
  3. Use the `RANK()` window function to assign a rank to each department based on the `EmployeeCount` in descending order.
  4. Group the results by `DepartmentID` and `Name` to ensure unique rows per department.
- **Assumptions**:
  - Each employee is associated with at least one department in `EmployeeDepartmentHistory`.
  - Departments with the same number of employees will share the same rank.
  
- **Example Output**:

| DepartmentID | Name            | EmployeeCount | DepartmentRank |
|--------------|-----------------|---------------|----------------|
| 1            | Sales           | 25            | 1              |
| 2            | Engineering     | 18            | 2              |
| 3            | Human Resources | 12            | 3              |
| 4            | Marketing       | 12            | 3              |
| 5            | Finance         | 8             | 5              |


In [14]:
SELECT d.DepartmentID, d.Name, COUNT(e.BusinessEntityID) AS EmployeeCount,
       RANK() OVER(ORDER BY COUNT(e.BusinessEntityID) DESC) AS DepartmentRank
FROM HumanResources.Department AS d
JOIN HumanResources.EmployeeDepartmentHistory AS e
ON d.DepartmentID = e.DepartmentID
GROUP BY d.DepartmentID, d.Name;


(16 rows affected)

Total execution time: 00:00:00.029

DepartmentID,Name,EmployeeCount,DepartmentRank
7,Production,180,1
3,Sales,18,2
5,Purchasing,13,3
10,Finance,11,4
11,Information Services,10,5
4,Marketing,10,5
1,Engineering,7,7
13,Quality Assurance,7,7
14,Facilities and Maintenance,7,7
15,Shipping and Receiving,6,10


### Proposition
Retrieve shifts from the `HumanResources.Shift` table that start after 9:00 AM.

### Functional Specification

#### Query Name
Shifts Starting After 9 AM

#### Description
This query selects shift details, including `ShiftID`, `Name`, `StartTime`, and `EndTime`, for all shifts that have a `StartTime` later than 9:00 AM.

#### Inputs
- `HumanResources.Shift` table, specifically the columns:
  - `ShiftID`: Unique identifier for each shift.
  - `Name`: Name of the shift.
  - `StartTime`: Start time of the shift.
  - `EndTime`: End time of the shift.

#### Outputs
- Columns returned:
  - `ShiftID`: Identifier of the shift.
  - `Name`: Name or label of the shift.
  - `StartTime`: The time the shift begins, filtered to only show times after 9:00 AM.
  - `EndTime`: The time the shift ends.

#### Steps
1. Select columns `ShiftID`, `Name`, `StartTime`, and `EndTime` from the `HumanResources.Shift` table.
2. Apply a filter in the `WHERE` clause to include only those rows where `StartTime` is later than `09:00:00`.

#### Assumptions
- `StartTime` is stored in a `time` format that allows direct comparison with the specified `09:00:00` value.
- Shifts with a start time exactly at 9:00 AM are excluded from the results.

#### Example Output
| ShiftID | Name         | StartTime | EndTime  |
|---------|--------------|-----------|----------|
| 2       | Afternoon    | 13:00:00  | 21:00:00 |
| 3       | Evening      | 15:00:00  | 23:00:00 |


In [3]:

SELECT ShiftID, Name, StartTime, EndTime
FROM HumanResources.Shift
WHERE StartTime > '09:00:00';

(2 rows affected)

Total execution time: 00:00:00.021

ShiftID,Name,StartTime,EndTime
2,Evening,15:00:00,23:00:00
3,Night,23:00:00,07:00:00
